## RAG

In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from langchain_community.document_loaders import TextLoader #load the document
from langchain_text_splitters import RecursiveCharacterTextSplitter #for creating chunks from the loaded document
from langchain_openai import OpenAIEmbeddings #for converting chunks into embeddings
from langchain_chroma import Chroma #database for stroring the embeddings

In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [12]:
import os
dir = os.getcwd()
db_dir = os.path.join(dir,"chroma_db")
print(db_dir)

c:\GaTech\Class Work\Semester 3\AI4DM\Git_H2\AI4DM_Lab\AI4DM_H2_Milestone 2_Viswak Raja - Copy\chroma_db


### Create vector DB

In [14]:
#Read the text content from the .txt file and load it as langchain document
loader = TextLoader('emotion.txt')
document = loader.load()

In [15]:
#Split the document into chunks using text splitters 
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(document)

print("Document chunk info:\n")
print(f"Number of document chunks: {len(chunks)}")
print(f"Sample chunk: \n{chunks[3].page_content}\n")

Document chunk info:

Number of document chunks: 247
Sample chunk: 
i guess it doesn t help that i got sick on black friday and was forced against my will to maintain my promise to stay in but being back in the city feels amazing
i feel as though i am on another adventure and i am more curious about it than anything else
i feel more amazed and more thankful for having e in our lives
i still sit back and feel amazed by the whole thing
i have been feeling overwhelmed with it all and needing to take time out
i replied feeling strange at giving the orders
i wonder if the homeowners would feel weird if i parked to gape at their landscaping
i both feel impatience at the rate of loss and impressed at the same time
i ran errands to buy cora a few newborn sized sleepers i had not previously made any newborn sized babies and went out to lunch to celebrate how great i was feeling i feel amazing no pain no pain meds and moving around almost completely normally at days out
i think or feel but like 

In [16]:
#create embeddings using openAI embeddings
embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small"
)

In [18]:
#store the embeddings and chunks into Chroma DB
Chroma.from_documents(documents=chunks, embedding=embeddings, persist_directory=db_dir)

### Retrieve and generate

In [19]:
#setting up the DB for retrieval
embeddings_used = OpenAIEmbeddings(model="text-embedding-3-small")
vectorDB = Chroma(persist_directory=db_dir,embedding_function=embeddings_used)

In [20]:
#setting up Retriver
retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})

In [21]:
def getRetriever(dir):
    """
    dir is the directory of the vector DB
    """
    embeddings_used = OpenAIEmbeddings(model="text-embedding-3-small")
    vectorDB = Chroma(persist_directory=dir,embedding_function=embeddings_used)
    retriever = vectorDB.as_retriever(search_type="similarity", search_kwargs={"k": 3})
    return retriever

In [53]:
def textGeneration_langChain_RAG(msg,type,retrieverDir, image_data):
    """
    msg is the scenario for the story from the pic (hugging face model output);
    type is the emotion of the story- Surprised, Anger, Joy, fear, Sadness
    retriever is the vector DB with relevant stories from txt 
    """
    llm = ChatOpenAI(
            model="gpt-4o",
            temperature=0.2,
            max_tokens=200,
            timeout=None,
            max_retries=2
        )



    prmpt = f'You are an expert at describing images for Blind and Low Vision users.Describe this image in {type} genre'
    out_message =  HumanMessagePromptTemplate.from_template(
        template=[
            {"type": "text", "text": prmpt},
            {
                "type": "image_url",
                "image_url": "{image_data}",
            },
        ]
    )

    prompt = ChatPromptTemplate.from_messages([out_message])

    
    rag_chain = prompt | llm | StrOutputParser()

    retriever = getRetriever(retrieverDir)

    out_message = rag_chain.invoke({
        "story_type" : type,
        "context":retriever,
        "image_data": image_data
        # "scenario_lang" : msg,
    })
    
    return out_message

def runModels_langchain_RAG(url, type, retrieverDir):
    image_data = local_image_to_data_url(url)
    scenario = img2text(url)
    story = textGeneration_langChain_RAG(scenario,type,retrieverDir, image_data)
    return([scenario,story])

import base64
from mimetypes import guess_type

# Function to encode a local image into data URL 
def local_image_to_data_url(image_path):
    mime_type, _ = guess_type(image_path)
    # Default to png
    if mime_type is None:
        mime_type = 'image/png'

    # Read and encode the image file
    with open(image_path, "rb") as image_file:
        base64_encoded_data = base64.b64encode(image_file.read()).decode('utf-8')

    # Construct the data URL
    return f"data:{mime_type};base64,{base64_encoded_data}"


In [52]:
scenario = "bookshelves with many different colored books on them in a library" #example output from huggingface model
story = textGeneration_langChain_RAG(scenario,"joy", db_dir)
print(story)

TypeError: textGeneration_langChain_RAG() missing 1 required positional argument: 'image_data'